In [1]:
cd ..

/home/abhishekabhishek/git/AQT-KimGroup


This notebook can be used to run different quantum circuits (e.g. GHZ, W) on IBMQ simulators or hardware, and construct a dataset of POVM measurements which can be used to perform quantum state tomography using attention-based models.

In [2]:
from qiskit import IBMQ, QuantumCircuit
import numpy as np
import os

from circuits import state_prep

%matplotlib inline

Setup access to IBM

In [3]:
# Setup account
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')

# Find available devices
devices = provider.backends(operational=True)
for x in devices:
    print(x.name(), x.configuration().n_qubits, x.status().pending_jobs)

ibmq_qasm_simulator 32 4
ibmq_lima 5 152
ibmq_belem 5 11
ibmq_quito 5 1
simulator_statevector 32 4
simulator_mps 100 4
simulator_extended_stabilizer 63 4
simulator_stabilizer 5000 4
ibmq_manila 5 26
ibm_nairobi 7 33
ibm_oslo 7 28


In [4]:
# Use one of the backends above
sim_backend = provider.backend.ibmq_qasm_simulator

Use the qiskit_experiments library to collect the tomography data

In [5]:
from qiskit_experiments.library import StateTomography

In [6]:
# System setup
n_qubits = 4
n_shots = 100

# Initialize the circuit
circ = QuantumCircuit(n_qubits)

# Choose which type of state to prepare
# One of ghz, w, bisep, random
state = 'w'

In [7]:
if state == 'ghz':
    circ.h(0)
    for idx in range(n_qubits-1):
        circ.cx(idx, idx+1)
elif state == 'w':
    if n_qubits == 3:
        circ.ry(1.9106332362490184, 0)
        circ.cu((np.pi/2), (np.pi/2), (np.pi/2), (np.pi/2), 0, 1)
        circ.cx(1, 2)
        circ.cx(0, 1)
        circ.x(0)
    else:
        circ = state_prep.create_w_state(n_qubits)
elif state == 'bisep':
    raise NotImplementedError
elif state == 'random':
    for idx in range(n_qubits):
        circ.h(idx)
        
# Draw the circuit
circ.draw() 

┌───┐┌─────┐       ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤0    ├───────┤ X ├┤ X ├┤ X ├
     └───┘│  CF │┌─────┐└─┬─┘└─┬─┘└─┬─┘
q_1: ─────┤1    ├┤0    ├──■────┼────┼──
          └─────┘│     │       │    │  
q_2: ────────────┤1 w3 ├───────■────┼──
                 │     │            │  
q_3: ────────────┤2    ├────────────■──
                 └─────┘

In [8]:
# No. of tomography circuits to draw
n_draw = 3

# POVM to use
povm = 'pauli6'

# Initalize the tomography circuits
qst_exp = StateTomography(circ)
qst_data = qst_exp.run(sim_backend, seed_simulation=100, 
                       shots=n_shots).block_for_results()

# Draw the tomogrpahy circuits
for circuit in qst_exp.circuits()[:n_draw]:
    print(circuit.draw())

     ┌───┐┌─────┐       ┌───┐┌───┐┌───┐ ░ ┌────────────┐ ░ ┌─┐         
q_0: ┤ X ├┤0    ├───────┤ X ├┤ X ├┤ X ├─░─┤ PauliMeasZ ├─░─┤M├─────────
     └───┘│  CF │┌─────┐└─┬─┘└─┬─┘└─┬─┘ ░ ├────────────┤ ░ └╥┘┌─┐      
q_1: ─────┤1    ├┤0    ├──■────┼────┼───░─┤ PauliMeasZ ├─░──╫─┤M├──────
          └─────┘│     │       │    │   ░ ├────────────┤ ░  ║ └╥┘┌─┐   
q_2: ────────────┤1 w3 ├───────■────┼───░─┤ PauliMeasZ ├─░──╫──╫─┤M├───
                 │     │            │   ░ ├────────────┤ ░  ║  ║ └╥┘┌─┐
q_3: ────────────┤2    ├────────────■───░─┤ PauliMeasZ ├─░──╫──╫──╫─┤M├
                 └─────┘                ░ └────────────┘ ░  ║  ║  ║ └╥┘
c_0: ═══════════════════════════════════════════════════════╩══╬══╬══╬═
                                                               ║  ║  ║ 
c_1: ══════════════════════════════════════════════════════════╩══╬══╬═
                                                                  ║  ║ 
c_2: ═══════════════════════════════════════════════════════════

In [9]:
# Get the results from the experiments
counts_dict = {}
header_dict = {}

idx = 0
for job_id in qst_data.job_ids:
    result = sim_backend.retrieve_job(job_id).result()
    counts, exp_results = result.get_counts(), result.results
    
    for count, exp_result in zip(counts, exp_results):
        counts_dict[idx] = count
        header_dict[idx] = exp_result.header.name.split('_')[1][1:-1].split(', ')
        idx+= 1

In [10]:
counts_dict[0], header_dict[0]

({'0001': 31, '0010': 29, '0100': 22, '1000': 18}, ['0', '0', '0', '0'])

Map the outcomes of the QST experiments into sequences that can be used to train the Transformer model

In [11]:
# No. of measurement sets for Pauli-6 POVM
n_sets = 3**n_qubits

data = np.zeros((n_sets*n_shots, n_qubits), dtype=int)
idx = 0

for n_set in range(n_sets):
    # Raw measurement counts for 1 experiment e.g. {'0...0':X, '1...1':Y}
    meas_dict = counts_dict[n_set]
    
    # Measurement setup e.g. ['0', ..., '0'] -> [Z, ..., Z]
    meas_setting = header_dict[n_set]
    meas_setting.reverse()
    
    for bitstring, count in meas_dict.items():
        # Map the projective outcomes to POVM outcomes
        # e.g. given setting [1...], map outcome [0..] to [2..] and [1..] to [3..]
        outcome = np.array([2*int(meas_setting[i]) + int(bitstring[i]) for i in range(n_qubits)])
        data[idx:idx+count] = outcome
        idx += count

print('No. of measurements in the dataset', len(data))

No. of measurements in the dataset 8100


Save the generated dataset to disk

In [12]:
# save options
overwrite = False
data_dir = 'data'
state_dir = data_dir + f'/{state}_{n_qubits}'

if not os.path.isdir(data_dir):
    print(f'{data_dir} does not exist. Creating the directory.')
    os.mkdir(f'{data_dir}')
    
if not os.path.isdir(state_dir):
    print(f'{state_dir} does not exist. Creating the directory.')
    os.mkdir(f'{state_dir}')
    
if os.path.exists(f'{state_dir}/{povm}_{len(data)}.npy'):
    if overwrite:
        print('overwriting existing dataset')
        np.save(f'{state_dir}/{povm}_{len(data)}.npy', data)
else:
    print('creating new dataset')
    np.save(f'{state_dir}/{povm}_{len(data)}.npy', data)

data/w_4 does not exist. Creating the directory.
creating new dataset
